In [18]:
#--------------------------------------------------------------------------------------------------------------------------------------
# LIBRARIES
#--------------------------------------------------------------------------------------------------------------------------------------

from ipyleaflet import Map, basemaps, Polyline, Marker, CircleMarker, LayerGroup, Polygon
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import HTML, Dropdown, IntRangeSlider, FloatRangeSlider, Output, Text, Image, VBox, HBox, Label, IntProgress
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from functools import partial
import numpy as np
import scipy.io as sio
from scipy import interpolate as interp
import time, datetime
from math import comb, exp

#--------------------------------------------------------------------------------------------------------------------------------------
# HEADER
#--------------------------------------------------------------------------------------------------------------------------------------
header = HTML("<center><h1>Volcano-Meteo-Tsurrogate Model v1.0</h1></center>", layout=Layout(height='auto',width='99%'))

#--------------------------------------------------------------------------------------------------------------------------------------
# lEFT PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#--------------
## instructions
#--------------

instruction_txt = Text(value='Select a Volcano', 
                       placeholder='', 
                       description='', 
                       style=dict(text_color='blue'), 
                       disabled=True,
                       layout=Layout(width='99%')
                      ) 
#-----
## map
#-----

m=Map(basemap=basemaps.Esri.WorldImagery, center=[0.0,0.0], zoom=1)
m.layout.width='auto'
m.layout.height='auto'

#-----------------
## Volcano Locations
#-----------------

# small offset in degrees for the triangle size
offset = 2.0  
# Askja
lat = 65.0111
lon = -16.7485
coord_askja = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_askja)
# Campi Flegrei
lat = 40.827
lon = 14.139
coord_campi = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_campi)
# Cotopaxi
lat = -0.6838
lon = -78.4372
coord_cotopaxi = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_cotopaxi)
# Cumbre Vieja
lat = 28.5728
lon = -17.8375
coord_cumbre = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_cumbre)
# Hunga Tonga - Hunga Ha apai
lat = -20.5700
lon = -175.3800
coord_hunga = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_hunga)
# Katla
lat = 63.6467
lon = -19.1303
coord_katla = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_katla)
# Saint Helens
lat = 46.2
lon = -122.18
coord_helens = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_helens)
# Pinatubo
lat = 15.1429
lon = 120.3496
coord_pinatubo = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_pinatubo)
# Popocatepetl
lat = 19.0224
lon = -98.6279
coord_popocatepetl = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_popocatepetl)
# Sakurajima
lat = 31.5833
lon = 130.6500
coord_Sakurajima = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_Sakurajima)
# Vesuvius
lat = 40.8224
lon = 14.4289
coord_vesuvius = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_vesuvius)
# Yellowstone
lat = 44.4123
lon = -110.7232
coord_yellowstone = Polygon(locations=[(lat + offset, lon),(lat - offset, lon - offset),(lat - offset, lon + offset)],color="white",fill_color="white",fill_opacity=0.8,draggable=False)
m.add_layer(coord_yellowstone)

#--------------------------------------------------------------------------------------------------------------------------------------
# MIDDLE PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#------------------------
## CLEAR selector: RESET
#------------------------

CLEAR_selector = Button(description='CLEAR', 
                     disabled=False,
                     button_style='danger',
                     tooltip='Reset all results',
                     icon=''
                    )
CLEAR_selector_out = Output()
def update_CLEAR(b):
    with CLEAR_selector_out:        
        CLEAR_selector_out.clear_output()
        m.center=[0.0, 0.0]
        m.zoom=1        
        instruction_txt.value = 'Select a Volcano'
        volcan_selector.value=''
        volcan_selector.disabled=False
        amplitude_selector.value=[16.0, 18.0] 
        amplitude_selector.disabled=True
        wavelength_selector.value=[550.0, 650.0] 
        wavelength_selector.disabled=True
        speed_selector.value=[310.0,320.0] 
        attenuation_selector.disabled=True
        attenuation_selector.value=[4.0,6.0] 
        attenuation_selector.disabled=True        
        param_box.layout.display = 'none'
        OK_selector.disabled=True
        OK_box.layout.display = 'none'
        OK_selector_out.clear_output()
        progress.value = 0        
        fig_zeta_box.layout.display = 'none'
        fig_velo_box.layout.display = 'none'
        fig_time_box.layout.display = 'none'
        stats_box.layout.display = 'none'
        stat_ele1_txt.value=''
        stat_ele2_txt.value=''
        stat_ele3_txt.value=''
        stat_ele4_txt.value=''
        stat_ele5_txt.value=''
        stat_ele6_txt.value=''
        stat_time1_txt.value=''
        stat_time2_txt.value=''
        stat_time3_txt.value=''
        stat_time4_txt.value=''
        stat_time5_txt.value=''
        stat_time6_txt.value=''
        loc_box.layout.display = 'none'
        loc_city_txt.value = ''
        loc_lon_txt.value = ''
        loc_lat_txt.value = ''
        if layer_group is not None:
            m.remove_layer(layer_group)
        for name, coord in volcan_loc.items():
            m.remove_layer(coord)
            coord.fill_color = "white"
            m.add_layer(coord)     
CLEAR_selector.on_click(update_CLEAR)

#-------------------------
## landslide zone selector
#-------------------------

volcan_selector = Dropdown( options = ('','Askja','Campi Flegrei','Cotopaxi','Cumbre Vieja','Hunga Tonga - Hunga Ha apai','Katla','Saint Helens','Pinatubo','Popocatepetl','Sakurajima','Vesuvius','Yellowstone'), 
                            layout=Layout(width='auto'))
volcan_out = Output()
volcan_loc = {
    "Askja": coord_askja,
    "Campi Flegrei": coord_campi,
    "Cotopaxi": coord_cotopaxi,
    "Cumbre Vieja": coord_cumbre,
    "Hunga Tonga - Hunga Ha apai": coord_hunga,
    "Katla": coord_katla,
    "Saint Helens": coord_helens,
    "Pinatubo": coord_pinatubo,
    "Popocatepetl": coord_popocatepetl,
    "Sakurajima": coord_Sakurajima,
    "Vesuvius": coord_vesuvius,
    "Yellowstone": coord_yellowstone
}
volcan_lon = {
    "Askja": -16.7485,
    "Campi Flegrei": 14.139,
    "Cotopaxi": -78.4372,
    "Cumbre Vieja": -17.8375,
    "Hunga Tonga - Hunga Ha apai": -175.3800,
    "Katla": -19.1303,
    "Saint Helens": -122.18,
    "Pinatubo": 120.3496,
    "Popocatepetl": -98.6279,
    "Sakurajima": 130.6500,
    "Vesuvius": 14.4289,
    "Yellowstone": -110.7232
}
volcan_lat = {
    "Askja": 65.0111,
    "Campi Flegrei": 40.827,
    "Cotopaxi": -0.6838,
    "Cumbre Vieja": 28.5728,
    "Hunga Tonga - Hunga Ha apai": -20.5700,
    "Katla": 63.6467,
    "Saint Helens": 46.2,
    "Pinatubo": 15.1429,
    "Popocatepetl": 19.0224,
    "Sakurajima": 31.5833,
    "Vesuvius": 40.8224,
    "Yellowstone": 44.4123
}
volcan_id = {
    "Askja": 0,
    "Campi Flegrei": 9,
    "Cotopaxi": 1,
    "Cumbre Vieja": 2,
    "Hunga Tonga - Hunga Ha apai": 3,
    "Katla": 4,
    "Saint Helens": 5,
    "Pinatubo": 6,
    "Popocatepetl": 7,
    "Sakurajima": 8,
    "Vesuvius": 9,
    "Yellowstone": 10
}
def update_loc(change):
    global coeff, lon_coast, lat_coast, name_coast
    with volcan_out:
        volcan_out.clear_output()
        selected = change['new']
        for name, coord in volcan_loc.items():
            m.remove_layer(coord)  # remove old
            if name == selected:
                coord.color = "red"
                coord.fill_color = "red"
            else:
                coord.color = "white"
                coord.fill_color = "white"
            m.add_layer(coord) 
        # Interpolation rule from distance to coordinates & Zoom map to selected zone
        if selected in volcan_loc:
            coord = volcan_loc[selected]
            lat = volcan_lat[selected]
            lon = volcan_lon[selected]
            id_simu = volcan_id[selected]
            m.center = [lat,lon]
            m.zoom = 3
            volcan_selector.disabled=True  
            amplitude_selector.disabled=False
            wavelength_selector.disabled=False
            speed_selector.disabled=False
            attenuation_selector.disabled=False
            param_box.layout.display = 'flex'
            OK_selector.disabled=False
            OK_box.layout.display = 'flex'
            lon_coast = sio.loadmat('data/coastal_locations.mat')['loc_lon']
            lat_coast = sio.loadmat('data/coastal_locations.mat')['loc_lat']
            name_coast = sio.loadmat('data/coastal_locations.mat')['loc_name']
            coeff = sio.loadmat('data/coastal_locations_coeff_hat.mat')['coeff_hat']
            coeff = coeff[id_simu]
            instruction_txt.value = 'Select the Atmospheric Pressure Parameters or Push CLEAR'
        else:
            m.center=[0.0, 0.0]
            m.zoom=1
            instruction_txt.value = 'Select a Volcano'
volcan_selector.observe(update_loc, names='value')

    
#-------------------
## amplitude selector
#-------------------

amplitude_selector = IntRangeSlider(value= [16, 18], min=10, max=200, step=1, disabled=True, continuous_update=True, orientation='horizontal', readout=True, readout_format='.1f',
                                   layout=Layout(width='auto'))
#-------------------
## wavelength selector
#------------------- 

wavelength_selector = IntRangeSlider(value= [550, 650], min=300, max=900, step=1, disabled=True, continuous_update=True, orientation='horizontal', readout=True, readout_format='.1f',
                                   layout=Layout(width='auto'))
#-------------------
## speed selector
#-------------------

speed_selector = IntRangeSlider(value= [310, 320], min=280, max=340, step=1, disabled=True, continuous_update=True, orientation='horizontal', readout=True, readout_format='.1f',
                                   layout=Layout(width='auto'))
#-------------------
## attenuation selector
#-------------------

attenuation_selector = IntRangeSlider(value= [4, 6], min=1, max=12, step=1, disabled=True, continuous_update=True, orientation='horizontal', readout=True, readout_format='.1f',
                                   layout=Layout(width='auto'))

param_box = VBox([Label("Amplitude [hPa]:"), amplitude_selector,
                  Label("Wavelength [km]:"), wavelength_selector,
                  Label("Speed [m/s]:"), speed_selector,
                  Label("Attenuation [hour]:"), attenuation_selector,
                 ])
param_box.layout.display = 'none'

#------------------------
## OK selector: model run
#------------------------

OK_selector = Button(description='OK', 
                     disabled=True,
                     button_style='success',
                     tooltip='Run the Model',
                     icon=''
                    )
OK_selector_out = Output()
layer_group=None
def update_OK(b):
    with OK_selector_out:
        global PTHA_zeta, PTHA_velo, PTHA_time, layers, layer_group
        OK_selector_out.clear_output(wait=True)        
        amplitude_selector.disabled=True
        wavelength_selector.disabled=True
        speed_selector.disabled=True
        attenuation_selector.disabled=True
        OK_selector.disabled=True
        progress.value = 0
        a0 = np.array([amplitude_selector.value[0],wavelength_selector.value[0],speed_selector.value[0],attenuation_selector.value[0]])
        b0 = np.array([amplitude_selector.value[1],wavelength_selector.value[1],speed_selector.value[1],attenuation_selector.value[1]])
        aa = np.array([amplitude_selector.min,wavelength_selector.min,speed_selector.min,attenuation_selector.min])
        bb = np.array([amplitude_selector.max,wavelength_selector.max,speed_selector.max,attenuation_selector.max])
        nw0 = 1000
        maxdeg0 = 5
        start = time.time()
        PTHA_zeta, PTHA_velo, PTHA_time = surrogate_model_gauss_patterson_PTHA(maxdeg0, aa, bb, coeff, nw0, a0, b0, progress=progress)
        end = time.time()
        length = end - start
        print(str(nw0) + " model runs produced in " + str(length) + " s")
        # Initialize plots with location 0
        n_locs = coeff[0]['zeta'].shape[0]
        m.center = [sum(lat_coast.flatten())/len(lat_coast.flatten()), sum(lon_coast.flatten())/len(lon_coast.flatten())]
        m.zoom = 1
        layers = []
        for i in range(n_locs):
            lat = float(np.ravel(lat_coast[i])[0])
            lon = float(np.ravel(lon_coast[i])[0])
            markers = CircleMarker(location=(lat, lon), radius=8, color="black", fill_color="white", fill_opacity=0.8, draggable=False)
            markers.on_click(partial(on_marker_click, id=i))
            layers.append(markers)
        layer_group=LayerGroup(layers=layers)
        m.add_layer(layer_group)
        instruction_txt.value = "Click on the map to select locations or Push CLEAR"
        fig_zeta_box.layout.display = 'flex'
        fig_velo_box.layout.display = 'flex'
        fig_time_box.layout.display = 'flex'
        stats_box.layout.display = 'flex'
        val_zeta = PTHA_zeta[0,:]
        val_time = PTHA_time[0,:]
        create_plot_widget(fig_zeta, PTHA_zeta[0,:], 0, "Maximum Meteo-Tsunami Elevation [m]", color="blue")
        create_plot_widget(fig_velo, PTHA_velo[0,:], 0,"Maximum Meteo-Tsunami Speed [m/s]", color="green")
        create_plot_widget(fig_time, PTHA_time[0,:], 0, "Meteo-Tsunami Time of Arrival [hour]", color="purple")
        loc_box.layout.display = 'flex'
        loc_city_txt.value = str(name_coast[0][0][0]) 
        loc_lon_txt.value = str(lon_coast[0][0])
        loc_lat_txt.value = str(lat_coast[0][0])
        tot_zeta = len(val_zeta)
        zeta1 = np.sum(val_zeta <= 0.5) * 100 /tot_zeta
        zeta2 = np.sum((val_zeta > 0.5) & (val_zeta <= 1.0)) * 100 /tot_zeta
        zeta3 = np.sum((val_zeta > 1.0) & (val_zeta <= 3.0)) * 100 /tot_zeta
        zeta4 = np.sum((val_zeta > 3.0) & (val_zeta <= 5.0)) * 100 /tot_zeta
        zeta5 = np.sum((val_zeta > 5.0) & (val_zeta <= 10.0)) * 100 /tot_zeta
        zeta6 = np.sum(val_zeta > 10.0) * 100 /tot_zeta
        stat_ele1_txt.value=str(zeta1)
        stat_ele2_txt.value=str(zeta2)
        stat_ele3_txt.value=str(zeta3)
        stat_ele4_txt.value=str(zeta4)
        stat_ele5_txt.value=str(zeta5)
        stat_ele6_txt.value=str(zeta6)
        tot_time = len(val_time)
        time1 = np.sum(val_time <= 1.0) * 100 /tot_time
        time2 = np.sum((val_time > 1.0) & (val_time <= 2.0)) * 100 /tot_time
        time3 = np.sum((val_time > 2.0) & (val_time <= 3.0)) * 100 /tot_time
        time4 = np.sum((val_time > 3.0) & (val_time <= 5.0)) * 100 /tot_time
        time5 = np.sum((val_time > 5.0) & (val_time <= 10.0)) * 100 /tot_time
        time6 = np.sum(val_time > 10.0) * 100 /tot_zeta
        stat_time1_txt.value=str(time1)
        stat_time2_txt.value=str(time2)
        stat_time3_txt.value=str(time3)
        stat_time4_txt.value=str(time4)
        stat_time5_txt.value=str(time5)
        stat_time6_txt.value=str(time6)        
OK_selector.on_click(update_OK)
progress = IntProgress(
    value=0, 
    min=0, 
    max=100,
    description='',
    style={'bar_color': 'maroon'},
)
OK_box = HBox([OK_selector, progress, OK_selector_out])
OK_box.layout.display = 'none'
def surrogate_model_gauss_patterson_PTHA(maxdeg, aa, bb, coeff_hat, nw, ar, br, progress=None):      
    nmodes = len(ar)
    nx = len(coeff_hat[0]['zeta'])
    # Build Legendre polynomials up to maxdeg
    Le = [None] * (maxdeg + 1)
    Le[0] = np.array([1.0])         # L_0 = 1
    Le[1] = np.array([1.0, 0.0])    # L_1 = x
    for n in range(2, maxdeg + 1):
        Le[n] = ((2 * n - 1) / n) * np.concatenate((Le[n - 1], [0])) - ((n - 1) / n) * np.concatenate(([0, 0], Le[n - 2]))  
    # Generate random input  
    np.random.seed(0)
    seed_01 = np.random.rand(nmodes, nw)    
    # Rescale to intervals
    Zw = np.tile(ar, (nw,1)).T + np.tile((br - ar), (nw,1)).T * seed_01
    # Preallocate output
    zeta_temp = np.zeros((nx, nw))
    velo_temp = np.zeros((nx, nw))
    time_temp = np.zeros((nx, nw))
    # Total iterations for progress
    total_steps = (maxdeg + 1 - max(0, maxdeg - nmodes + 1))
    current_step = 0
    for alpha_norm1 in range(max(0, maxdeg - nmodes + 1), maxdeg + 1):
        # Smolyak coefficient
        C_alpha = ((-1)**(maxdeg - alpha_norm1) * comb(nmodes - 1, maxdeg - alpha_norm1))            
        # Retrieve coefficients
        alpha    = np.array(coeff_hat[alpha_norm1]['alpha']) 
        zeta_hat = np.array(coeff_hat[alpha_norm1]['zeta'])   
        velo_hat = np.array(coeff_hat[alpha_norm1]['velo'])
        time_hat = np.array(coeff_hat[alpha_norm1]['time_zeta'])
        nww = alpha.shape[0]            
        for l in range(nww):
            multH = np.ones(nw)
            for n in range(nmodes):
                x_scaled = (2 * Zw[n, :] - aa[n] - bb[n]) / (bb[n] - aa[n])
                multH *= np.polyval(Le[alpha[l, n]], x_scaled)
            for i in range(nx):
                zeta_temp[i, :] += C_alpha * zeta_hat[i, l] * multH
                velo_temp[i, :] += C_alpha * velo_hat[i, l] * multH
                time_temp[i, :] += C_alpha * time_hat[i, l] * multH
        # Update progress bar at each alpha_norm1 step
        current_step += 1
        if progress:
            progress.value = int(current_step / total_steps * progress.max)
    # Undo log transform
    zeta = np.exp(zeta_temp)
    velo = np.exp(velo_temp)
    time = np.exp(time_temp)
    return zeta, velo, time
# Create markers and add click events
def on_marker_click(event=None, id=None, **kwargs):
    loc_index = id
    loc_city_txt.value = str(name_coast[loc_index][0][0])
    loc_lon_txt.value = str(lon_coast[loc_index][0])
    loc_lat_txt.value = str(lat_coast[loc_index][0])  
    m.center = [lat_coast[loc_index][0],lon_coast[loc_index][0]]
    m.zoom = 5
    for i, marker in enumerate(layers):
        if i == id:
            marker.fill_color = "red"            
        else:
            marker.fill_color = "white"
    val_zeta = PTHA_zeta[loc_index,:]
    val_time = PTHA_time[loc_index,:]
    create_plot_widget(fig_zeta, PTHA_zeta[loc_index,:], 0, "Maximum Meteo-Tsunami Elevation [m]", color="blue")
    create_plot_widget(fig_velo, PTHA_velo[loc_index,:], 0, "Maximum Meteo-Tsunami Speed [m/s]", color="green")
    create_plot_widget(fig_time, PTHA_time[loc_index,:], 0, "Meteo-Tsunami Time of Arrival [hour]", color="purple")
    tot_zeta = len(val_zeta)
    zeta1 = np.sum(val_zeta <= 0.5) * 100 /tot_zeta
    zeta2 = np.sum((val_zeta > 0.5) & (val_zeta <= 1.0)) * 100 /tot_zeta
    zeta3 = np.sum((val_zeta > 1.0) & (val_zeta <= 3.0)) * 100 /tot_zeta
    zeta4 = np.sum((val_zeta > 3.0) & (val_zeta <= 5.0)) * 100 /tot_zeta
    zeta5 = np.sum((val_zeta > 5.0) & (val_zeta <= 10.0)) * 100 /tot_zeta
    zeta6 = np.sum(val_zeta > 10.0) * 100 /tot_zeta
    stat_ele1_txt.value=str(zeta1)
    stat_ele2_txt.value=str(zeta2)
    stat_ele3_txt.value=str(zeta3)
    stat_ele4_txt.value=str(zeta4)
    stat_ele5_txt.value=str(zeta5)
    stat_ele6_txt.value=str(zeta6)
    tot_time = len(val_time)
    time1 = np.sum(val_time <= 1.0) * 100 /tot_time
    time2 = np.sum((val_time > 1.0) & (val_time <= 5.0)) * 100 /tot_time
    time3 = np.sum((val_time > 5.0) & (val_time <= 10.0)) * 100 /tot_time
    time4 = np.sum((val_time > 10.0) & (val_time <= 15.0)) * 100 /tot_time
    time5 = np.sum((val_time > 15.0) & (val_time <= 20.0)) * 100 /tot_time
    time6 = np.sum(val_time > 20.0) * 100 /tot_zeta
    stat_time1_txt.value=str(time1)
    stat_time2_txt.value=str(time2)
    stat_time3_txt.value=str(time3)
    stat_time4_txt.value=str(time4)
    stat_time5_txt.value=str(time5)
    stat_time6_txt.value=str(time6)

#--------------------------------------------------------------------------------------------------------------------------------------
# RIGHT PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#-------------------------
## logo
#-------------------------

img_widget = Image(value=open("data/splash.png", "rb").read(), format='png', layout=Layout(width='60%'))

#-------------------------
## location information
#-------------------------

loc_header_txt =  HTML("<center><h3>Selected Location</h3></center>", layout=Layout(height='auto',width='99%'))
loc_city_txt = Text(value='', placeholder='',description='Place',disabled=True)
loc_lon_txt = Text(value='', placeholder='',description='Long. [°E]',disabled=True)
loc_lat_txt = Text(value='', placeholder='',description='Lat. [°N]',disabled=True)
loc_box = VBox([loc_header_txt, loc_city_txt, loc_lon_txt, loc_lat_txt])
loc_box.layout.display = 'none'

#--------------------------------------------------------------------------------------------------------------------------------------
# FOOTER
#--------------------------------------------------------------------------------------------------------------------------------------

#-------------------------
## figures
#-------------------------

fig_zeta = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_zeta.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_velo = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_velo.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_time = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_time.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_zeta_box = VBox([fig_zeta])
fig_velo_box = VBox([fig_velo])
fig_time_box = VBox([fig_time])
fig_zeta_box.layout.display = 'none'
fig_velo_box.layout.display = 'none'
fig_time_box.layout.display = 'none'

# pdf/cdf helper
def pdf_cdf(samples, bins=40):
    hist, edges = np.histogram(samples, bins=bins, density=True)
    centers = (edges[:-1] + edges[1:]) / 2
    sorted_s = np.sort(samples)
    cdf = np.arange(1, len(sorted_s)+1)/len(sorted_s)
    return centers, hist, sorted_s, cdf
# create a single FigureWidget
def create_plot_widget(fig, arr, option, title, color="blue"):
    x_pdf, y_pdf, x_cdf, y_cdf = pdf_cdf(arr)
    fig.data = []  # clear old traces
    #if option == 1:
    #    base = datetime.datetime(1970, 1, 1)
    #    x_pdf = [base + datetime.timedelta(hours=float(m)) for m in x_pdf]
    #    x_cdf = [base + datetime.timedelta(hours=float(m)) for m in x_cdf]
    fig.add_bar(x=x_pdf, y=y_pdf, marker_color=color, opacity=0.5, name="PDF")
    fig.add_scatter(x=x_cdf, y=y_cdf, mode='lines', line=dict(color="red", width=2),
                    name="CDF", yaxis="y2")
    fig.update_layout(
        title=title,
        yaxis=dict(title="PDF"),
        yaxis2=dict(title="CDF", overlaying="y", side="right"),
        template="simple_white",
    )
    #if option == 1:
    #    fig.update_xaxes(type="date", tickformat="%H:%M:%S")

#-------------------------
## statistics
#-------------------------

stat_header_txt = HTML("<center><h3>Probability [%]</h3></center>", layout=Layout(height='auto',width='99%'))

stat_ele_header_txt =  HTML("<h4>Elevation [m]</h4>", layout=Layout(height='auto',width='auto'))
stat_ele1_txt = Text(value='', placeholder='',description='P(<0.5)',disabled=True)
stat_ele2_txt = Text(value='', placeholder='',description='P(0.5-1.0)',disabled=True)
stat_ele3_txt = Text(value='', placeholder='',description='P(1.0-3.0)',disabled=True)
stat_ele4_txt = Text(value='', placeholder='',description='P(3.0-5.0)',disabled=True)
stat_ele5_txt = Text(value='', placeholder='',description='P(5.0-10.0)',disabled=True)
stat_ele6_txt = Text(value='', placeholder='',description='P(>10.0)',disabled=True)
stat_ele_box = VBox([stat_ele_header_txt, stat_ele1_txt, stat_ele2_txt, stat_ele3_txt, stat_ele4_txt, stat_ele5_txt, stat_ele6_txt])

stat_time_header_txt =  HTML("<h4>Time of Arrival [hour]</h4>", layout=Layout(height='auto',width='auto'))
stat_time1_txt = Text(value='', placeholder='',description='P(<1)',disabled=True)
stat_time2_txt = Text(value='', placeholder='',description='P(1-5)',disabled=True)
stat_time3_txt = Text(value='', placeholder='',description='P(5-10)',disabled=True)
stat_time4_txt = Text(value='', placeholder='',description='P(10-15)',disabled=True)
stat_time5_txt = Text(value='', placeholder='',description='P(15-20)',disabled=True)
stat_time6_txt = Text(value='', placeholder='',description='P(>20)',disabled=True)
stat_time_box = VBox([stat_time_header_txt, stat_time1_txt, stat_time2_txt, stat_time3_txt, stat_time4_txt, stat_time5_txt, stat_time6_txt])

stats_box = VBox([stat_header_txt, HBox([stat_ele_box, stat_time_box])])
stats_box.layout.display = 'none'

#--------------------------------------------------------------------------------------------------------------------------------------
# FINAL LAYOUT
#--------------------------------------------------------------------------------------------------------------------------------------

AppLayout(header=header,
          left_sidebar=VBox([instruction_txt,m],layout=Layout(display='flex',justify_content='center')), 
          center=VBox([CLEAR_selector, CLEAR_selector_out,
                       Label("Volcano:"), volcan_selector, volcan_out,                       
                       param_box,
                       OK_box],
                      layout=Layout(display='flex',justify_content='center')),
          right_sidebar=VBox([img_widget,loc_box],layout=Layout(display='flex', align_items='center')),
          footer=HBox([VBox([fig_zeta_box, fig_velo_box]), VBox([fig_time_box, stats_box])],layout=Layout(display='flex', justify_content='center')),
          pane_widths=[3, 3, 3],
          pane_heights=[1, 6, 10]
         )

AppLayout(children=(HTML(value='<center><h1>Volcano-Meteo-Tsurrogate Model v1.0</h1></center>', layout=Layout(…